# **GR5242 HW01 Problem 2: Dropout as a form of regularization**


**Instructions**: This problem is an individual assignment -- you are to complete this problem on your own, without conferring with your classmates.  You should submit a completed and published notebook to Courseworks; no other files will be accepted.


**Description**: In this exercise we will try to understand regularizing effects of [Dropout](https://jmlr.org/papers/v15/srivastava14a.html) method which was initially introduced in the deep learning context to mitigate overfitting, though we intend to study its behavior as a regularizer in a rather simpler settting.


## Regression
Indeed, linear models correspond to a one layer neural networks with linear activation. Denote $f_{\beta}:\mathbb{R}^p \to \mathbb{R}: x \mapsto \sum_{j=1}^{p} \beta_j x_j$ to represent the output of such network. Given $n$ samples $(x_i,y_i)_{i \leq n} \in (\mathbb{R}^{p} \times \mathbb{R})^n$ we want to regress the response onto the observed covariates using the following MSE loss:

$$ L(\beta) = \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2 $$

In the current atmosphere of deep learning practice, it is rather popular to have moderately large networks in order to learn a task (we will see more on this later in the course). This corresponds to having $p \gg n$ in our setting which allows more flexibility in our linear model. However, in these cases where the model can be too complicated one can use `explicit` regularization to penalize complex models. One way to do so is ridge regression:

$$ \hat{\beta}_{\lambda} = \arg\min_{\beta \in \mathbb{R}^p}{ L(\beta) + \lambda \sum_{j=1}^{p}\beta_j^2 } $$


**Question 1**: Show that $\hat{\beta}_{\lambda} = (X^{T}X + \lambda I_{p \times p})^{-1} X^{T} Y$ where $X = (x_1^{'},\dots,x_n^{'})^{'} \in \mathbb{R}^{n \times p}$ and $Y = (y_1,\dots,y_n)^{'} \in \mathbb{R}^{n}$.  

**Your Answer Here**:

We have $L(\beta) = (Y - X \beta)^T (Y - X \beta) = (Y^T - \beta^T X^T) (Y - X \beta) = Y^T Y - Y^T X \beta - \beta^T X^T Y + \beta^T X^T X \beta = Y^T Y - 2 Y^T X \beta+ \beta^T X^T X \beta$. Let 
$$ \hat{L}(\beta) = L(\beta) + \lambda \beta^T \beta = Y^T Y - 2 Y^T X \beta + \beta^T X^T X \beta + \lambda \beta^T \beta$$

We want to minimize $\hat{L}(\beta)$, so we differentiate w.r.t $\beta$:
$$ \nabla \hat{L}(\beta) = -2 X^T Y + 2 X^T X \beta + 2 \lambda \beta = 2 \left( -X^T Y + X^T X \beta + \lambda I_{p \times p} \beta \right) $$
Solving for $\nabla \hat{L}(\beta) = 0$:
$$ X^T Y = X^T X \beta + \lambda I_{p \times p} \beta = \left( X^T X + \lambda I_{p \times p} \right) \beta $$
This yields
$$ \hat{\beta_\lambda} = \left( X^T X + \lambda I_{p \times p} \right)^{-1} (X^T Y) $$

## Dropout

We now present the connection between dropout method and ridge regression (outlined in more detail in [Wager et al.](https://arxiv.org/pdf/1307.1493.pdf))

To recap, dropout randomly drops units along with their input/output connections. We now want to apply this method to our simple setting. Let us define the indicator random variable $I_{ij}$ to be whether the $j$'th neuron is present or not in predicting the response of the $i$'th sample. More explicitly, the ouput of the network for $i$'th sample becomes $\sum_{j=1}^{p} I_{ij}\beta_j x_{ij}$ where

$$ I_{ij} = \begin{cases} 0 & \text{with probability } \phi \\ \frac{1}{1-\phi} & \text{with probability } 1-\phi  \end{cases}, \quad 1 \leq j \leq p,\, 1\leq i \leq n, $$

drawn independently from the training dataset. Note that $\mathbb{E}[I_{ij}] = 1$, thus the output of the network is $f_{\beta}(x_i)$ on average.



**Question 2**: Write down explicitly the loss function after using the dropout as a function of $I = (I_{ij})_{i \leq n, j \leq p}$ denoted by $L(\beta, I)$.

**Your Answer Here**:

We can write the $i$-th row of the matrix $ \left( X_{n \times p} \odot I_{n \times p} \right) \, \beta_{p \times 1}$:
$$ \sum_{j=1}^{p} I_{ij}\beta_j x_{ij} = ((X \odot I) \beta)_i$$
where $(X \odot I)$ is the element-wise product, instead of a matrix multiplication, so $(X \odot I)$ has a dimension of $n \times p$. Then, the loss function is:

$$ L(\beta, I) = (Y - ((X \odot I) \beta)^T (Y - ((X \odot I) \beta) $$
or
$$ L(\beta, I) = Y^T Y - 2 Y^T (X \odot I) \beta + \beta^T (X \odot I)^T (X \odot I) \beta $$

It can be shown that SGD + Dropout is in some sense equivalent to minimizing the loss function $L(\beta,I)$ on average. Related to this point, the following problem justifies why dropout can be thought as a form of regularization.

**Question 3**: Suppose the feature matrix $X \in \mathbb{R}^{n \times p}$ have standardized features (norm of each column is one). Show that the solution to the following problem corresponds to a ridge regression with $\lambda = \frac{\phi}{1-\phi}$.
$$ \arg\min_{\beta \in \mathbb{R}^p} \mathbb{E}[L(\beta,I)]$$
where the expectation is over the randomness of indicator random variables.

**Hint**: You can assume that taking derivative can pass through expectation.

**Your Answer Here**:





Use the fact that $X$, $Y$, $\beta$ are nonrandom with respect to $I$, and that $\mathbb{E}[I] = 1$, so $\mathbb{E}[I \odot X] = X$.

Moreover, $(I \odot X)^T (I \odot X)$ is a $p \times p$ matrix whose $(i,j)$-th entry is given by
$$(I \odot X)^T (I \odot X) = \left( \sum_{k=1}^{n} I_{ik} X_{ik} I_{kj} X_{kj} \right)_{i,j}$$

Note that $\mathbb{E}[ I_{i_1,j_1} I_{i_2, j_2} ] = (1-\phi) \left( \frac{1}{1 - \phi} \right)^2 + \phi 0^2 = \frac{1}{1 - \phi} $ if $i_1=i_2$ and $j_1 = j_2$. Otherwise, the expectation is zero. Only the diagonal elements in $\left( \sum_{k=1}^{n} I_{ik} X_{ik} I_{kj} X_{kj} \right)_{i,j}$ contain elements with $i_1=i_2$ and $j_1 = j_2$. That is, for $d=1,...,p$,
$$ \mathbb{E}\left[ \left( \sum_{k=1}^{n} I_{dk} X_{dk} I_{kd} X_{kd} \right)_{d,d} \right] = \mathbb{E}\left[ I_{dd}^2 X_{dd}^2 \right] = \frac{1}{1-\phi} X_{dd}^2 = \frac{1}{1-\phi} \left( X^T X \right)_{d,d} $$
We can conclude that
$$ \mathbb{E}\left[ (I \odot X)^T (I \odot X) \right] = \frac{1}{1-\phi} 1_{p} X^T X $$
where $1_{p}$ is the $p \times p$ identity matrix.

$$ \mathbb{E}[L(\beta, I)] = Y^T Y - 2 Y^T \mathbb{E} \left[ I \odot X \right] \beta + \beta^T \mathbb{E} \left[ (I \odot X)^T (I \odot X) \right] \beta $$
$$ = Y^T Y - 2 Y^T X \beta + \left( \frac{1}{1- \phi} \right) \beta^T X^T X \beta $$
$$ = Y^T Y - 2 Y^T X \beta + \left( 1 + \frac{\phi}{1- \phi} \right) \beta^T X^T X \beta $$
$$ = Y^T Y - 2 Y^T X \beta + \beta^T X^T X \beta + \frac{\phi}{1- \phi} \beta^T X^T X \beta $$
Moreover, since $X$ is normalized, $X^T X \approx I$ and we have
$$ = Y^T Y - 2 Y^T X \beta + \beta^T X^T X \beta + \frac{\phi}{1- \phi} \beta^T \beta $$
This is indeed a Rigde regression with $\lambda = \frac{\phi}{1- \phi}$.